# Исключения (exceptions)

Подробнее - Мартелли А.,.. - Python. Справочник стр. 201

## Инструкция `try` / `except` / `finally`

### `try` / `except`
Эта форма инструкции содержит одно или несколько предложений `except` и необязательное
предложение `else`. Тело каждого блока `except` называется обработчиком исключений.

In [ ]:
try:
    a = 1 + '2'
except:
    print('Ошибка типов')   # обработчик исклбючения

Ошибка типов


In [ ]:
# общий случай
import sys
x = 1; y = 0; z = '2'

try:
    x / y   # заменить y на z, затем на x
except ZeroDivisionError as target:
    # target - идентификатор переменной которую Python связывает с объектом
    # исключения непосредственно перед выполнением обработчика исключения
    print('Попытка деления на 0')
    print(sys.exc_info())       # получить текущий объект исключения
    print(ZeroDivisionError)
    print(f'{target = }')
except:
    print('Произошла некоторая ошибка')
else:       # Необязательное предложение else инструкции try / except
    print('Инструкция выполняется только если исключений не произошло')
finally:
    print('Иструкция выполняется в любом случае')

Попытка деления на 0
(<class 'ZeroDivisionError'>, ZeroDivisionError('division by zero'), <traceback object at 0x000002CDBE5EDA00>)
<class 'ZeroDivisionError'>
target = ZeroDivisionError('division by zero')
Иструкция выполняется в любом случае


__try__ содержит код, в котором отслеживается возникновение исключения.

__except__ - обработчик исключения, в инструкции try может содержаться несколько инструкций except - для разных видов исключений (частных или общих).

__else__ - содержит код, который выполняется только в случае успешного выполнения предложения try.

__finally__ - устанавливает так называемый обработчик очистки. Код этого обработчика выполняется всегда, независимо от того, каким образом было завершено выполнение предложения try. Код внутри части finally обычно освобождает критически важный ресурс или восстанавливает временно измененное состояние.

Обработчики специфических исключений всегда должны предшествовать обработчикам
более общих исключений. Несоблюдение этого правила приведет к тому, что обработчики
более специфических исключений никогда не смогут быть выполнены.

Избегайте использования "пустых" предложений except (т.е. предложений except без выражения), если в них не возбуждается вручную какое-либо исключение: такой небрежный стиль может затруднить поиск причин возникновения ошибки, поскольку подобные инструкции слишком общие и могут маскировать ошибки в коде и другие виды логических ошибок.

## Генерация исключения

In [ ]:
raise TypeError("This is exception")

TypeError: This is exception

# Контекстные менеджеры и блоки `with`

In [ ]:
# Самый распространенный пример – гарантированное закрытие объекта файла:

# open() возвращает объект класса TextIOWrapper, который является контекстным менеджером.
# в начале блока with будет вызван метод __enter__ объект класса TextIOWrapper
with open('Python\\environment.txt', encoding='utf-8') as file:
    # file будет ссылаться на объект, возвращаемый методом __enter__ контекстного
    # мененджера - в данном случае - это self, т.е. сам объект, возвращаемый open():
    print(type(file))                   # объект контекстного мнеджера TextIOWrapper
    lst = [line for line in file]       # записать все строки в список

# при выходе из конструкии with вызывается метод __exit__ контекстного менеджера. В данном
# случае, метод __exit__ класса TextIOWrapper обеспечивает закрытие файла.

print('first line:', lst[0])

# Переменная file все еще доступна для чтения атрибутов:
print(file.closed, file.encoding)
# Но выполнить операцию ввода-вывода для fp по завершении блока with нельзя, т.к. уже был
# вызван метод TextIOWrapper.__exit__ при покидании блока with и файл закрыт.

<class '_io.TextIOWrapper'>
first line: 

True utf-8


Инструкция with - это воплощение в Python известной идиомы С++ "получение
ресурса есть инициализация"

https://ru.wikipedia.org/wiki/Получение_ресурса_есть_инициализация

Предложение `with` было задумано, для того чтобы упростить конструкцию `try/finally`, гарантирующую, что некоторая операция будет выполнена после блока, даже если этот блок прерван в результате исключения, предложения return или вызова `sys.exit()`.
Объекты контекстных менеджеров служат для управления предложением with, точно так же, как итераторы управляют предложением for. Протокол контекстного менеджера состоит из методов __enter__ и __exit__. В начале блока with вызывается метод __enter__ контекстного менеджера. А роль части finally играет обращение к методу __exit__ контекстного менеджера в конце блока with.

Часть as в предложении with необязательна. В случае open она необходима, чтобы
получить ссылку на файл, но некоторые контекстные менеджеры возвращают None за
неимением чего-то полезного.

Подробнее: Рамально Лучано - Python к вершинам мастерства с. 482

In [ ]:
# контекстные менеджеры предназначены для выполнения заключительных
# (или предаварительных) операций независимо от того, возникло ли
# исключение на этапе выполнения основного действия.

# создадим свой класс контекстного менеджера:

class TraceBlock:
    def message(self, arg):             # некий метод класса
        print('running', arg)
    def __enter__(self):                # это инструкция, которая
        print('starting with block')    # запускается до выполнения блока
        return self

    # а этот метод запускается после выполнения блока в любом случае,
    # независимо от того, было исключение или нет. В качестве аргументов
    # она принимает тип исключения, значение и диагностическую инфу (если
    # исключения не возникало, то эти аргументы передается значение None)
    def __exit__(self, exc_type, exc_value, exc_tb):
        if exc_type is None:
            print('exited normally\n')
        else:
            print('raise an exception!', exc_type)
            return False

with TraceBlock() as action:    # объект класса TraceBlock присваивается
                                # переменной после as (если оно имеется)
    action.message('test 1')    # здесь нет исключений и мы
    print('reached')            # достигаем конца блока

with TraceBlock() as action:
    action.message('test 2')
    raise TypeError             # возникает исключение и последняя
    print('not reached')        # инструкция блока не достигается

starting with block
running test 1
reached
exited normally

starting with block
running test 2
raise an exception! <class 'TypeError'>


TypeError: 

# `sorted()`

In [ ]:
# функция sorted
s = ['This', 'is', '-', 'some', 'text']
print(s)
print(sorted(s))                    # упорядочивание в лексикографическом порядке
print(sorted(s, reverse=True))      # по убыванию
# key передается функция, по результатам которой должно быть выполнено сравнение
print(sorted(s, key=len))           # упорядочивание по длине элемента